<a href="https://colab.research.google.com/github/Annmodels/mnist_recon/blob/master/Denoising_gray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import zipfile
import random
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from urllib import request
from keras.models import Sequential
from keras.layers import Dense


In [0]:
!pip install kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d zalando-research/fashionmnist 

In [0]:
from zipfile import ZipFile
fn = "fashionmnist.zip"
with ZipFile(fn,'r')as zip1:
  zip1 = zip1.extractall()

In [0]:
img_database = np.loadtxt('fashion-mnist_train.csv',delimiter =',',skiprows = 1)[:,1:]
print(img_database.shape)
total_num_images = (img_database.shape[0])

In [0]:
n_input = 784
hidden1 = 620
hidden2 = 448
hidden3 = 224
hidden4 = 112
hidden5 = 56
hidden6 = 28
hidden7 = 28
hidden8 = 56
hidden9 = 112
hidden8 = 224
hidden9 = 448
hidden10 = 620
output_layer = 784

In [0]:
learning_rate = 0.05
epochs = 30
batch_size = 100

In [0]:
x = tf.placeholder(tf.float32,[None,n_input])
y = tf.placeholder(tf.float32,[None,output_layer])

In [0]:
weights = {"w1": tf.Variable(tf.random_normal([n_input,hidden1])),
           "w2": tf.Variable(tf.random_normal([hidden1,hidden2])),
           "w3": tf.Variable(tf.random_normal([hidden2,hidden3])),
           "w4": tf.Variable(tf.random_normal([hidden3,hidden4])),
           "w5": tf.Variable(tf.random_normal([hidden4,hidden5])),
           "w6": tf.Variable(tf.random_normal([hidden5,hidden6])),
           "w7": tf.Variable(tf.random_normal([hidden6,hidden7])),
           "w8": tf.Variable(tf.random_normal([hidden7,hidden8])),
           "w9": tf.Variable(tf.random_normal([hidden8,hidden9])),
           "w10":tf.Variable(tf.random_normal([hidden9,hidden10])),
           "w11": tf.Variable(tf.random_normal([hidden10,output_layer]))
           }
biases = {"b1":tf.Variable(tf.random.normal([hidden1])),
          "b2":tf.Variable(tf.random.normal([hidden2])),
          "b3":tf.Variable(tf.random.normal([hidden3])),
          "b4":tf.Variable(tf.random.normal([hidden4])),
          "b5":tf.Variable(tf.random.normal([hidden5])),
          "b6":tf.Variable(tf.random.normal([hidden6])),
          "b7":tf.Variable(tf.random.normal([hidden7])),
          "b8":tf.Variable(tf.random.normal([hidden8])),
          "b9":tf.Variable(tf.random.normal([hidden9])),
          "b10":tf.Variable(tf.random.normal([hidden10])),
          "b11":tf.Variable(tf.random.normal([output_layer]))
          }



In [0]:
z1 = tf.nn.sigmoid(tf.add(tf.matmul(x,weights["w1"]),biases["b1"]))
z2 = tf.nn.sigmoid(tf.add(tf.matmul(z1,weights["w2"]),biases["b2"])) 
z3 = tf.nn.sigmoid(tf.add(tf.matmul(z2,weights["w3"]),biases["b3"]))
z4 = tf.nn.sigmoid(tf.add(tf.matmul(z3,weights["w4"]),biases["b4"])) 
z5 = tf.nn.sigmoid(tf.add(tf.matmul(z4,weights["w5"]),biases["b5"])) 
z6 = tf.nn.sigmoid(tf.add(tf.matmul(z5,weights["w6"]),biases["b6"])) 
z7 = tf.nn.sigmoid(tf.add(tf.matmul(z6,weights["w7"]),biases["b7"])) 
z8 = tf.nn.sigmoid(tf.add(tf.matmul(z7,weights["w8"]),biases["b8"]))
z9 = tf.nn.sigmoid(tf.add(tf.matmul(z8,weights["w9"]),biases["b9"]))
z10 = tf.nn.sigmoid(tf.add(tf.matmul(z9,weights["w10"]),biases["b10"]))
nn_output = tf.add(tf.matmul(z10,weights["w11"]),biases["b11"])

In [0]:
#z1 = tf.layers.dense(x,hidden1,activation = tf.nn.sigmoid)
#3": tf.Variable(tf.random_normal([hiddenhidden3])) = tf.layers.dense(z1,hidden2,activation = tf.nn.sigmoid)
#3 = tf.layers.dense(z2,hidden3,activation = tf.nn.sigmoid)
#1 = tf.nn.sigmoid(tf.add(tf.matmul(x,weights["w1"]),biases["b1"]))  = tf.layers.dense(z3,hidden4,activation = tf.nn.sigmoid)
#5 = tf.layers.dense(z4,hidden5,activation = tf.nn.sigmoid)
# = tf.nn.sigmoid(tf.add(tf.matmul(x,weights["w1"]),biases["b1"]))  = tf.layers.dense(z5,hidden6,activation = tf.nn.sigmoid)
#7 = tf.layers.dense(z6,hidden7,activation = tf.nn.sigmoid)
#1 = tf.nn.sigmoid(tf.add(tf.matmul(x,weights["w1"]),biases["b1"])) = tf.layers.dense(z7,hidden8,activation = tf.nn.sigmoid)
#n_output = tf.layers.dense(z8,output_layer)

In [0]:
np.random.shuffle(img_database)
x_train = img_database
#normalize
#x_train = x_train
x_train_noisy = x_train + 10*np.random.normal(0,1,size = x_train.shape)
plt.imshow(x_train[0].reshape(28,28),cmap = 'gray')
plt.show()
plt.imshow(x_train_noisy[0].reshape(28,28),cmap = 'gray')
plt.show()

In [0]:
#print(nn_output)
#print(y)
closs = tf.reduce_mean(tf.square(nn_output-y))
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(closs)
init = tf.global_variables_initializer()

In [0]:
x_actual = x_train[:10]
noisy_image = x_train_noisy[:10]
with tf.Session() as sess:
  sess.run(init)
  for epoch in range(epochs):
    for i in range(int(total_num_images/batch_size)):
      x_epoch = x_train[i*batch_size:(i+1)*batch_size]
      x_noise_epoch = x_train_noisy[i*batch_size:(i+1)*batch_size]
      _,loss = sess.run([optimizer,closs],feed_dict = {x:x_noise_epoch,y:x_epoch})
    print("epoch",epoch,'/',epochs,'loss:',loss)
  denoised = sess.run(nn_output,feed_dict = {x:noisy_image})
  #print(denoised.shape)

In [0]:
#import zipfile 
fig,axes = plt.subplots(nrows = 3,ncols =10,sharex = True,sharey =True,figsize = (20,4))
for images,row in zip([x_actual,noisy_image,denoised],axes):
  for img,ax in zip(images,row):
    ax.imshow(img.reshape((28,28)),cmap = 'gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
fig.tight_layout(pad = 0.1)
